# **School District Aid Data**





# Dataset Cleaning

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Lottery_Aid_to_Education__Beginning_2002 (3).csv to Lottery_Aid_to_Education__Beginning_2002 (3).csv


In [ ]:
df = pd.read_csv("Lottery_Aid_to_Education__Beginning_2002 (3).csv")
df

,Beginning Fiscal Year,Ending Fiscal Year,County,School District,Amount of Aid
0,2002,2003,Albany,Albany,"7,242,297.04"
1,2002,2003,Albany,Berne-Knox-Westerlo,"874,046.94"
2,2002,2003,Albany,Bethlehem,"2,786,954.71"
3,2002,2003,Albany,Cohoes,"1,820,297.94"
4,2002,2003,Albany,Green Island,"241,859.93"
...,...,...,...,...,...
12267,2019,2020,Wyoming,Perry,"1,460,861.27"
12268,2019,2020,Wyoming,Warsaw,"1,539,093.10"
12269,2019,2020,Wyoming,Wyoming,"197,556.57"
12270,2019,2020,Yates,Dundee,"1,116,357.36"


In [ ]:
df_copy = df.copy()

#remove School District and Ending Fiscal Year
df_copy = df_copy.drop(columns=['School District', 'Ending Fiscal Year'])

#assign 'Beginning Fiscal Year' from int64 to datetime
df_copy['Beginning Fiscal Year'] = pd.to_datetime(df_copy['Beginning Fiscal Year'], format='%Y')

#remove spaces from "Amount of Aid" column name 
df_copy = df_copy.rename(columns = {' Amount of Aid ':'Amount of Aid'})

#rename the misspelled county name 'Saint Lawrence' with 'St Lawrence'
df_copy['County'] = df_copy["County"].replace({'Saint Lawrence':'St Lawrence'})

#groupy by county and year, sum aid amounts, and return a dataframe
df_copy = df_copy.groupby(['County', 'Beginning Fiscal Year']).sum().reset_index()


#add Year column
df_copy["Year"] = pd.to_datetime(df_copy["Beginning Fiscal Year"]).dt.year

#df_copy.info() #---->(1044, 4) dataframe (18 years x 58 NYcounties=1044)
df_copy['County'].unique()


array(['Albany', 'Allegany', 'Broome', 'Cattaraugus', 'Cayuga',
       'Chautauqua', 'Chemung', 'Chenango', 'Clinton', 'Columbia',
       'Cortland', 'Delaware', 'Dutchess', 'Erie', 'Essex', 'Franklin',
       'Fulton', 'Genesee', 'Greene', 'Hamilton', 'Herkimer', 'Jefferson',
       'Lewis', 'Livingston', 'Madison', 'Monroe', 'Montgomery', 'Nassau',
       'New York City', 'Niagara', 'Oneida', 'Onondaga', 'Ontario',
       'Orange', 'Orleans', 'Oswego', 'Otsego', 'Putnam', 'Rensselaer',
       'Rockland', 'Saratoga', 'Schenectady', 'Schoharie', 'Schuyler',
       'Seneca', 'St Lawrence', 'Steuben', 'Suffolk', 'Sullivan', 'Tioga',
       'Tompkins', 'Ulster', 'Warren', 'Washington', 'Wayne',
       'Westchester', 'Wyoming', 'Yates'], dtype=object)

In [ ]:
df_copy2019 = df_copy[(df_copy['Year'].isin([2018,2019]))]
df_copy2019 = df_copy2019.rename(columns = {'County':'county_name'})

#df_copy2019.info() #---->(116,4) dataframe (2 years x 58 NYcounties=1044)

In [ ]:
df_rank_2018copy = df_rank_2019.copy()
df_rank_2019copy = df_rank_2020.copy()

#assign years to each row
df_rank_2018copy['Year']= 2018
df_rank_2019copy['Year']= 2019

#select relevant columns for new database
df_rank_2019copy = df_rank_2019copy.iloc[: , [2,26,16,19,11,12,13,18]]
df_rank_2018copy = df_rank_2018copy.iloc[: , [2,26,16,19,11,12,13,18]]

#already checked that columns are the same, so will merge
df_rank_combine = pd.concat([df_rank_2018copy, df_rank_2019copy])

#rename the misspelled county name 'St. Lawrence' with 'St Lawrence'
df_rank_combine["county_name"] = df_rank_combine["county_name"].replace({'St. Lawrence':'St Lawrence'})


In [ ]:
#Keep only counties in NYC
county_nyc=['New York','Kings','Bronx','Richmond','Queens']
NYCrank=df_rank_combine[df_rank_combine['county_name'].isin(county_nyc)]


In [ ]:
d2019 = {'county_name': ['New York','Kings','Bronx','Richmond','Queens'],
         'Year':[2019,2019,2019,2019,2019],
         'Pop_25_44_2019': [595749,828246,407631,123892,666351],
         'county_pop_2019': [1628706,2559903,1418207,476143,2253858],
         'receive_lunch_aid2019':[108039, 231919,183034,34637,196515]}
df_pop2019 = pd.DataFrame(data=d2019)
df_p2019c = pd.merge(df_pop2019, NYCrank, on=['county_name','Year'])

df_p2019c['Total_some_college'] = df_p2019c['%some_college']*df_p2019c['Pop_25_44_2019']/100
df_p2019c['Total_food_insecure'] = df_p2019c['%food_insecure']*df_p2019c['county_pop_2019']/100
df_p2019c['Total_lunch_aid'] = df_p2019c['%lunch_aid']*df_p2019c['receive_lunch_aid2019']/100
df_p2019c = df_p2019c.iloc[: , [2,3,4,7,10,11,12,13]]
df_p2019c.loc['Total2019']= df_p2019c.sum()
df_p2019c['county_name'] = "New York City"
df_p2019c['Year'] = 2019
df_p2019c.reset_index()
df_p2019c = df_p2019c.drop(labels=[0,1,2,3,4], axis=0)


df_p2019c['%food_insecure'] = df_p2019c['Total_food_insecure'] /df_p2019c['county_pop_2019']
df_p2019c['%lunch_aid'] = df_p2019c['Total_lunch_aid'] /df_p2019c['receive_lunch_aid2019']
df_p2019c['%some_college'] = df_p2019c['Total_some_college'] /df_p2019c['Pop_25_44_2019']

In [ ]:
d2018 = {'county_name': ['New York','Kings','Bronx','Richmond','Queens'],
         'Year':[2018,2018,2018,2018,2018],
         'Pop_25_44_2018': [597226,411292,834877,124205,679580],
         'county_pop_2018': [1628701,2582830,1432132,476179,2278906],
         'receive_lunch_aid2018':[110472, 238173,187293,34905,204916]}
df_pop2018 = pd.DataFrame(data=d2018)
df_p2018c = pd.merge(df_pop2018, NYCrank, on=['county_name','Year'])


df_p2018c['Total_some_college'] = df_p2018c['%some_college']*df_p2018c['Pop_25_44_2018']/100
df_p2018c['Total_food_insecure'] = df_p2018c['%food_insecure']*df_p2018c['county_pop_2018']/100
df_p2018c['Total_lunch_aid'] = df_p2018c['%lunch_aid']*df_p2018c['receive_lunch_aid2018']/100
df_p2018c = df_p2018c.iloc[: , [2,3,4,7,10,11,12,13]]
df_p2018c.loc['Total2018']= df_p2018c.sum()
df_p2018c['county_name'] = "New York City"
df_p2018c['Year'] = 2018
df_p2018c.reset_index()
df_p2018c = df_p2018c.drop(labels=[0,1,2,3,4], axis=0)

df_p2018c['%food_insecure'] = df_p2018c['Total_food_insecure'] /df_p2018c['county_pop_2018']*100
df_p2018c['%lunch_aid'] = df_p2018c['Total_lunch_aid'] /df_p2018c['receive_lunch_aid2018']*100
df_p2018c['%some_college'] = df_p2018c['Total_some_college'] /df_p2018c['Pop_25_44_2018']*100


In [ ]:
df_nyc = pd.concat([df_p2018c, df_p2019c]).reset_index()
df_nyc = df_nyc.iloc[: , [9,10,11,12,13,4,5]]
df_nyc['graduation_rate'] = 'NaN'
df_nyc = df_nyc.iloc[: , [0,1,2,3,5,7,4,6]]

In [ ]:
#sort by year and county, name, reset index
df_rank_combine = df_rank_combine.sort_values(['Year','county_name'],ascending = (True,True))

#reset index and drop existing index
df_rank_combine = df_rank_combine.reset_index(drop=True)

#remove the NYC counties
df_rank_final = df_rank_combine.drop(labels=[2,23,30,40,42,64,85,92,102,104], axis=0)


#merge on county_name and Year
df_final2 = pd.concat([df_rank_final, df_nyc]).reset_index()


#sort by year and county, name, reset index
df_final2 = df_final2.sort_values(['Year','county_name'],ascending = (True,True))

#reset index and drop existing index
df_final2 = df_final2.reset_index(drop=True)
df_final2.drop('index', axis=1, inplace=True) #--->(124,10) dataframe (2 years x 58 NYcounties=124)

#Combine Finacail aid and rank data, merge on county_name and Year
df_final = pd.merge(df_copy2019, df_final2, on=['county_name','Year'])

#sort by year and county, name, reset index
df_final = df_final.sort_values(['Year','county_name'],ascending = (True,True))

#reset index and drop existing index
df_final = df_final.reset_index(drop=True)


In [ ]:
df_final.info() #(116,8)----->#58counties X 2 years

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   county_name            116 non-null    object        
 1   Beginning Fiscal Year  116 non-null    datetime64[ns]
 2   Amount of Aid          116 non-null    float64       
 3   Year                   116 non-null    int64         
 4   %food_insecure         116 non-null    float64       
 5   %lunch_aid             116 non-null    float64       
 6   graduation_cohort      116 non-null    float64       
 7   graduation_rate        116 non-null    object        
 8   %some_college          116 non-null    float64       
 9   household_income       116 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 9.2+ KB


In [ ]:
df_final.head(5)

,county_name,Beginning Fiscal Year,Amount of Aid,Year,%food_insecure,%lunch_aid,graduation_cohort,graduation_rate,%some_college,household_income
0,Albany,2018-01-01,43828381.49,2018,12.2,35.448380,3014.0,84.4061,76.604723,65659.0
1,Allegany,2018-01-01,12114969.12,2018,12.4,55.555556,502.0,87.8486,59.542065,46491.0
2,Broome,2018-01-01,40800060.82,2018,13.0,52.820043,2205.0,82.9478,66.510187,47910.0
3,Cattaraugus,2018-01-01,21253961.21,2018,12.5,52.949158,1041.0,85.0144,55.056312,47875.0
4,Cayuga,2018-01-01,13858754.34,2018,10.6,47.259490,762.0,83.0709,60.276255,55460.0


df_copy has your original data. 
df_copy2019 has your original data with beginning year 2018 and 2019 only. 
df_final has your data and Laurynes combined but only for 2018/2019. Her NYC countines (Kings, Queens, New York, Bronx and Richmond) were combined to be able to merge with your data. The data use is buried in one of the cells above for reference.

The years are 2018 and 2019. Consider labeling 2018/2019 and 2019/2020. 
Data is limited for 2020 so I wouldn't discuss the COVID angle (covid quarantine started ~3/2020) since the data timeframe does not capture this time period. Can comment on seeing the "beginning" of a relationship but need more data.
As we discussed focus on financial aid/food insecurity and FA/education:
1)show line graphof FA to show trends over many years as an inrroduction but focus on 2018 and 2019.
2)barplots to show distribtjin of FA over time
3)check correlations. if see signal graph. 
4) Can seperate out 'New York City' data and analyze seperately.
5)graph FA vs household income to see if the money is going where needed.
6) graph amount of Aid vs %food insecure
7)graph amount of Aid vs %lunch aid
8) FA vs graduation rate
9) %food_insecure vs household income or 
10) any other graphs you think woud be interesting


In [ ]:
#prints list of number of times element appears in column
#df_countynum = df.groupby('County').size().reset_index(name='count')
#print (df_countynum)

#temporaly display 999 rows
#with pd.option_context('display.max_rows', 999):
#    print (df_new2)

#county_list = df['County'].nunique() #----->58
#county_list = df['County'].unique() #---->numpy.ndarray
#county_list = df['County'].unique().tolist #---->builtin_function_or_method
#county_list = pd.Series(df['County'].unique()) #---->pandas.core.series.Series
#type(county_list4)

In [ ]:
 #display 10 rows
pd.set_option('display.max_rows', df.shape[0]+1)

In [ ]:
#remove commas from amount of aid column
df['Amount of Aid'] = df[' Amount of Aid '].str.replace(',','')
#convert amount of aid to float
df['Amount of Aid'] = df['Amount of Aid'].astype(float)
#get rid of ' amount of aid ' column
df.drop([' Amount of Aid '], axis=1, inplace=True)

In [ ]:
#change years from int to str/object
df['Beginning Fiscal Year'] = df['Beginning Fiscal Year'].astype('str')
df['Ending Fiscal Year'] = df['Ending Fiscal Year'].astype('str')

In [ ]:
df

In [ ]:
#starting at 2011
df_starting_2011 = df
df_starting_2011 = df_starting_2011.drop(df.index[range(6154)])
df_starting_2011

In [ ]:
#starting at 2017
df_starting_2017 = df
df_starting_2017 = df_starting_2017.drop(df.index[range(10233)])
df_starting_2017

In [ ]:
#starting at 2018
df_starting_2018 = df
df_starting_2018 = df_starting_2018.drop(df.index[range(10912)])
#2018-2019
df_2018_to_2019 = df_starting_2018.drop(df_starting_2018.index[-681:])
df_2018_to_2019


In [ ]:
#2019-2020
df_2019_to_2020 = df
df_2019_to_2020 = df_2019_to_2020.drop(df.index[range(11591)])
df_2019_to_2020

In [ ]:
#2018-2020 for tableau
df_starting_2018 = df
df_starting_2018 = df_starting_2018.drop(df.index[range(10912)])
df_starting_2018

In [ ]:
from google.colab import files

df_starting_2011.to_csv('df_starting_2011.csv')
files.download('df_starting_2011.csv')

In [ ]:
from google.colab import files

df_starting_2017.to_csv('df_starting_2017.csv')
files.download('df_starting_2017.csv')

In [ ]:
from google.colab import files

df_starting_2018.to_csv('df_starting_2018.csv')
files.download('df_starting_2018.csv')

In [ ]:
#Export/download df_2018_to_2019 dataframe as csv file
from google.colab import files

df_2018_to_2019.to_csv('df_2018_to_2019.csv')
files.download('df_2018_to_2019.csv')

In [ ]:
#Export/download df_2019_to_2020 dataframe as csv file
from google.colab import files

df_2019_to_2020.to_csv('df_2019_to_2020.csv')
files.download('df_2019_to_2020.csv')

In [ ]:
from google.colab import files

df_final.to_csv('df_final.csv')
files.download('df_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Lauryne's data sets

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving rank2019.csv to rank2019.csv


In [ ]:
df_rank_2019 = pd.read_csv("rank2019.csv")
df_rank_2019

,fips,state,county_name,health_outcomes_r,health_factors_r,length_life_r,quality_life_r,yppl,%poor_fair_health,%adult_obesity,food_environment_index,graduation_cohort,graduation_rate,%some_college,life_expectancy,%diabetes,%food_insecure,%limited_access,household_income,%lunch_aid,%african_american,%american_indian,%asian,%pacific_islander,%hispanic,%white
0,36087,New York,Rockland,1,6,3,3,4272.900333,14.089078,23.7,8.8,3248,84.544335,63.805615,83.312875,8.8,9.2,3.337587,87873,37.106402,11.477857,0.525135,6.495311,0.135921,17.967391,62.692022
1,36059,New York,Nassau,2,1,5,13,4300.055841,12.252958,23.2,9.7,16554,90.769603,74.845911,82.827126,8.5,5.6,0.468288,107465,28.178347,11.560597,0.532890,10.133595,0.095362,17.150610,59.851451
2,36119,New York,Westchester,3,4,2,19,4144.305550,14.281145,20.7,9.2,11163,89.554779,71.819318,83.525185,7.9,7.7,1.379185,90483,36.438612,13.732703,0.950172,6.528477,0.126805,24.863197,53.383545
3,36091,New York,Saratoga,4,2,6,4,4906.755021,11.160446,26.4,8.9,2742,90.919037,77.947014,80.920521,9.3,7.9,3.984430,82732,22.913952,1.694443,0.232306,2.949941,0.050899,3.230971,90.299692
4,36061,New York,New York,5,12,1,30,3984.712120,14.532237,15.8,8.3,16235,74.536495,83.735332,84.516308,7.0,13.3,0.000000,84133,64.594362,12.372359,1.183858,12.983210,0.182432,26.130531,46.677563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,36063,New York,Niagara,58,52,61,59,7710.310077,15.846324,33.3,8.2,2431,84.821061,67.137244,77.627084,12.2,12.0,4.237825,54317,48.095221,7.010429,1.136149,1.135675,0.041168,2.946131,85.537648
58,36013,New York,Chautauqua,59,55,62,56,8008.553112,18.089410,33.1,7.9,1521,83.497699,58.008628,77.461471,11.7,12.4,6.750473,45840,53.865533,2.144972,0.727648,0.700525,0.050370,7.566294,87.382794
59,36009,New York,Cattaraugus,60,61,54,61,7397.155204,18.529141,36.5,8.2,1041,85.014409,55.056312,77.761956,12.8,12.5,2.583937,47875,52.949158,1.450587,3.471324,0.840358,0.031029,2.089259,90.498785
60,36105,New York,Sullivan,61,50,60,60,7685.394052,19.036735,32.4,8.3,742,78.436658,54.399772,78.136145,11.3,11.0,4.669431,52504,57.680945,8.167186,0.703451,1.691727,0.079486,16.195271,71.826191


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving rank2020.csv to rank2020.csv


In [ ]:
df_rank_2020 = pd.read_csv("rank2020.csv")
df_rank_2020

,fips,state,county_name,health_outcomes_r,health_factors_r,length_life_r,quality_life_r,yppl,%poor_fair_health,%adult_obesity,food_environment_index,graduation_cohort,graduation_rate,%some_college,life_expectancy,%diabetes,%food_insecure,%limited_access,household_income,%lunch_aid,%african_american,%american_indian,%asian,%pacific_islander,%hispanic,%white
0,36001,New York,Albany,25,10,15,44,5821.072418,14.208671,27.3,8.0,3014,84.406105,76.783305,80.033193,9.1,12.0,4.779735,64536,35.501123,12.793821,0.296630,6.783734,0.067401,6.100932,71.807487
1,36003,New York,Allegany,47,52,49,34,7283.136814,14.416292,35.9,8.1,502,87.848606,61.366776,77.999828,10.9,12.4,2.991699,46262,51.937743,1.277191,0.316606,1.294422,0.040922,1.677795,94.294637
2,36005,New York,Bronx,62,62,36,62,6695.768724,26.798947,29.0,7.7,14750,67.511864,51.169291,80.559175,12.5,16.0,0.000920,38566,86.602547,29.222586,2.913558,4.500074,0.414766,56.365475,9.130723
3,36007,New York,Broome,52,37,51,49,7415.065526,15.929985,28.0,8.0,2205,82.947846,67.255530,78.673505,10.6,12.8,3.240672,50928,54.397057,5.469610,0.268185,4.427134,0.066785,4.341565,82.996885
4,36009,New York,Cattaraugus,57,59,47,60,7254.951597,16.370700,37.6,8.1,1041,85.014409,56.038590,77.848799,13.3,12.5,2.583937,48017,50.843238,1.378188,3.547631,0.871942,0.037741,2.123894,90.393024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,36115,New York,Washington,41,38,42,37,6949.619936,14.258736,33.6,8.4,690,79.565217,55.048419,78.567357,12.4,10.5,4.286443,55913,46.449938,2.939687,0.328447,0.558851,0.034315,2.849813,92.297008
58,36117,New York,Wayne,40,33,45,26,7146.866044,15.446684,36.3,8.6,1164,87.199313,64.357821,78.694902,12.1,9.8,3.488927,61118,46.736886,2.802452,0.416371,0.696172,0.041082,4.473486,89.829455
59,36119,New York,Westchester,5,3,2,23,4257.651377,14.801189,20.7,9.1,11163,89.554779,71.893480,83.505387,7.6,7.7,1.379185,94521,36.182755,13.804293,0.956478,6.435534,0.132284,25.140346,53.088118
60,36121,New York,Wyoming,18,28,25,9,6456.009504,13.954072,37.9,8.2,339,93.805310,55.188175,78.742584,12.3,10.5,6.066659,57709,43.360071,5.119122,0.444056,0.563802,0.029936,3.260571,89.901459


#EDA

In [ ]:
df_2018_to_2019.info()

In [ ]:
df_2019_to_2020.info()

In [ ]:
df_2018_to_2019.describe() #put year as a string or object or datetime

In [ ]:
df_2019_to_2020.describe()

In [ ]:
x = df_2018_to_2019['County']
y = df_2018_to_2019['Amount of Aid']

fig = plt.figure(figsize = (25, 10))

plt.bar(x, y, color ='maroon',
        width = 2)
 
plt.xlabel("County")
plt.ylabel("Amount of Aid")
plt.title("Amount of Aid per County from 2018-2019")
plt.show()

In [ ]:
x = df_2019_to_2020['County']
y = df_2019_to_2020['Amount of Aid']

fig = plt.figure(figsize = (25, 10))

plt.bar(x, y, color ='maroon',
        width = 2)
 
plt.xlabel("County")
plt.ylabel("Amount of Aid")
plt.title("Amount of Aid per County from 2019-2020")
plt.show()

In [ ]:
a = df_2018_to_2019['Amount of Aid']
y = df_2019_to_2020['Amount of Aid']

fig = plt.figure(figsize = (25, 10))

plt.bar(x, y, color ='maroon',
        width = 2)
 
plt.xlabel("County")
plt.ylabel("Amount of Aid")
plt.title("Amount of Aid per County from 2019-2020")
plt.show()

In [ ]:
sns.boxplot(x = df["Beginning Fiscal Year"], y= df["Amount of Aid"], data = df)
plt.title("Amount of Aid at Beginning Fiscal Year")
plt.figure(figsize = (20, 10))


In [ ]:
import pingouin